<img src="images/logo.png" width="200">

# KogSys-KI-B - Assignment 2

### Adversarial Search, Constraint Satisfaction Problems

_Abgabefrist: **15.06.2025**_

---



#### Abgabe Informationen

Laden Sie Ihre Lösung über den VC-Kurs hoch. Bitte laden Sie **ein Zip-Archiv** pro Gruppe hoch. Dieses muss enthalten:

- Ihre Lösung als **Notebooks** (pro Gesamtaufgabe eine `.ipynb`-Datei)
- Ein Ordner mit dem Namen **images**, der alle Ihre Bilder enthält, falls Sie welche verwendet haben (halten Sie die Bildgrößen relativ klein)

Ihr Zip-Archiv sollte wie folgt benannt werden:

```
assignment_<Assignmentnummer>_solution_<Gruppennummer>.zip
```

In dieser Aufgabe können Sie insgesamt **30 Punkte** erreichen. Von diesen Punkten werden **3 Bonuspunkte** für die Prüfung wie folgt berechnet:

| **Points in Assignment** | **Bonus Points for Exam** |
| :----------------------: | :-----------------------: |
|            30            |             3             |
|            25            |            2.5            |
|            20            |             2             |
|            15            |            1.5            |
|            10            |             1             |
|            5             |            0.5            |

<div class='alert alert-block alert-danger'>

##### **Wichtige Hinweise**

1. **Diese Aufgabe wird benotet. Sie können Bonuspunkte für die Prüfung erwerben.**
2. **Wenn offensichtlich ist, dass eine Aufgabe von einer anderen Quelle kopiert wurde und keine eigenständige Arbeit geleistet wurde, werden keine Bonuspunkte vergeben. Bitte formulieren Sie alle Antworten in Ihren eigenen Worten!**
3. **Falls LLMs (wie ChatGPT oder Copilot) zur Erstellung Ihrer Einreichung verwendet wurden, geben Sie dies bitte gemäß den gängigen wissenschaftlichen Praktiken an. Siehe auch die [KI Policy im VC-Kurs](https://vc.uni-bamberg.de/mod/page/view.php?id=1980835)**

### Setup

Um euer assignment aufzusetzen, müsst ihr die notwendigen pakete installieren, welche in der Datein `requirements.txt` gelistet sind. Dies könnt ihr machen, indem ihr die folgende Zelle ausführt.

In [36]:
# Installiert die benötigten Pakete mit dem akutell ausgewählten Python-Interpreter
%pip install -U -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Aufgabe 2 - Bedingungserfüllungsprobleme (CSP) am Beispiel "Hashiwokakero"

*Für insgesamt <mark>15</mark> Punkte*

In dieser Aufgabe ist es das Ziel, einen **Constraint Satisfaction Problem (CSP) Solver** für das [Hashiwokakero Puzzel-Spiel](https://de.wikipedia.org/wiki/Hashiwokakero) zu implementieren. Eure Aufgabe ist es, die CSP-Solver Implementierung vervollständigen.

### CSP Formulierung:
* **Variables (X)**: Potentielle Brücken zwischen benachbarten Inseln
* **Domains (D)**: Anzahl Brücken (0, 1, oder 2) zwischen Inselpaaren
* **Constraints (C)**:
  - Jede Insel muss exakt die durch ihre Zahl angegebene Anzahl Brücken haben
  - Brücken können sich nicht kreuzen
  - Brücken können nur horizontal oder vertikal verlaufen
  - Alle Inseln müssen verbunden sein um ein einziges Netzwerk zu bilden



### Bibliotheks-Imports

In der folgenden Zelle werden einige wichtige Bibliotheken importiert. Dies soll hier kurz erläutert werden. Verwenden Sie keine anderen Drittanbieterbibliotheken.

- `collections.deque` und `collections.defaultdict`: Diese werden für die Implementierung der Datenstrukturen verwendet, die für die Konnektivitätsprüfung verwendet wird.
- `time`: Wird für den Abbruch des CSP-Solvers nach maximaler Laufzeit verwendet.
- `typing.List`, `typing.Tuple`, `typing.Dict`, `typing.Optional` und `typing.Iterable`: Wird für Typanmerkungen in den Methodenspezifikationen benötigt.
- `pprint`: Wird für die Ausgabe von Datenstrukturen in einem lesbareren Format verwendet.


**_In der nächsten Codezelle muss nichts geändert werden._**

In [37]:
from collections import deque, defaultdict
import time
from typing import List, Tuple, Dict, Optional, Iterable
from pprint import pprint

## Vorgegebene Hilfsfunktionen

**_An den Hilfsfunktionen muss nichts verändert werden. Versuchen Sie aber alle nachzuvollziehen, um sie entsprechend zu nutzen._**

### 0. Datenstrukturen und Spielbrett

- Gegebene Datenstrukturen des Spielbretts:
  - Die `Island` enthält die Position und notwendige Anzahl an Brücken für eine Insel. 
  - Das `HashiwokakeroBoard` ist die Implementierung des Spielbretts. Es enthält Logik, um:
    - eine Insel von spezifischen Koordinaten zu erhalten (`get_island`),
    - Brücken hinzuzufügen (`add_bridge`),
    - alle Möglichen Verbindungen zwischen Inseln zu erhalten (`find_potential_connections`)
  - Die `HashiwokakeroGameEngine` implementiert die Anzeige des Spiels und Interaktionen mit pygame:
    - `solve` im Konstruktor ist eine Funktion die das CSP Problem lösen soll. 
    - `run` startet das Spiel und zeigt das Spielbrett an.\
      In diesem Modus sind mehrere Shortcuts verfügbar, die unter [Spielaufruf](#spielaufruf) beschrieben sind.
    - `board` ist das aktuelle Spielbrett.
    - `load_puzzle` lädt ein Puzzle nach Name (`puzzle1`, `puzzle2` oder `puzzle3`) oder mit einer Liste an Inseln im Format `list[tuple[row: int,col: int,value: int]]`.
- Datenstrukturen für den CSP-Solver:
  - **`type Var_PossibleBridge`**: Repräsentiert eine mögliche Brücke zwischen zwei Inseln. (Variable `x` des CSPs)
  - **`domain`**: alle möglichen Werte für eine Variable (Anzahl der Brücken zwischen zwei Inseln, 0, 1 oder 2).
  - **`type Assignment`**: Repräsentiert eine Zuordnung von Variablen zu Werten. Es ist ein Dictionary, das die Variablen (`Var_PossibleBridge`) den möglichen Brücken zuordnet. Der Wert ist aus der `domain` oder `None`, wenn die Variable noch nicht zugeordnet ist. \
    Hilfsfunktionen für das Arbeiten mit `Assignment`:
    - `get_bridge_count`: Ermittelt die Anzahl der Brücken für eine `Island` in einem `Assignment`.
    - `get_unassigned_vars`: Gibt eine Liste aller unzugeordneten Variablen in einem `Assignment` zurück. Optional nur für eine bestimmte `Island`.

Beispiel:
```python
#                row, col, value
island1 = Island(0,   0,   2    )
island2 = Island(0,   1,   1    ) 
island3 = Island(1,   0,   1    )
board = HashiwokakeroBoard(islands=[island1, island2, island3])

assignment: Assignment = {
    (island1, island2): 1, # 1 bridge from island1 to island2
    (island1, island3): None # no bridge count assigned yet
}
```

In [38]:
from hashiwokakero import Island, HashiwokakeroBoard, HashiwokakeroGameEngine

type Var_PossibleBridge = Tuple[Island, Island]

domain = (0, 1, 2)

type Assignment = Dict[Var_PossibleBridge, Optional[int]]

### Generelle Hilfsfunktionen ###

def get_bridge_count(island: Island, assignment: Assignment) -> int:
    """
    Gibt die Anzahl der Brücken zurück, die aktuell für eine Insel zugewiesen sind.
    """
    return sum(val for var, val in assignment.items() if island in var and val is not None)

def get_unassigned_vars(assignment: Assignment, island: Optional[Island] = None) -> List[Var_PossibleBridge]:
    """
    Gibt eine Liste aller Variablen zurück, die unzugewiesen sind (und optional zu der angegebenen Insel gehören).

    Args:
        assignment (Assignment): Aktuelle Zuweisung der Variablen.
        island (Optional[Island], optional): Wenn gegeben alle unzugewiesenen Variablen mit der gegebenen Insel. Defaults to None.
    """
    return [var for var, val in assignment.items() if island is None or (island in var and val is None)]

### 1. Prüfen der Insel-Erfüllbarkeit

Berechnet die aktuelle Brückenanzahl einer Insel für ein Assignment und prüft ob das Inselziel noch erreichbar ist.



In [39]:

def is_island_satisfiable(island: Island, assignment: Assignment) -> bool:

    current_bridges = get_bridge_count(island, assignment)

    # Kann geforderte Brückenanzahl nicht überschreiten
    if current_bridges > island.value:
        return False

    remaining_needed = island.value - current_bridges

    # Bereits erfüllt
    if remaining_needed == 0:
        return True

    unassigned_vars = get_unassigned_vars(assignment, island)

    # Prüfen ob Ziel noch erreichbar ist
    max_possible_additional = len(unassigned_vars) * max(domain)

    return max_possible_additional >= remaining_needed


### 2. Überprüfung ob eine neue Brücke erstellt werden kann

Prüft ob eine neue Brücke zwischen zwei Inseln erstellt werden kann, ohne eine bestehende Brücke zu kreuzen.

Nutzt eine Hilfsfunktion `do_bridges_cross` um zu prüfen, ob zwei Brücken sich kreuzen.


In [40]:
def do_bridges_cross(bridge1: Var_PossibleBridge, bridge2: Var_PossibleBridge) -> bool:

    (island1_1, island1_2) = bridge1
    (island2_1, island2_2) = bridge2

    # Prüfen ob eine horizontal und eine vertikal ist
    bridge1_horizontal = (island1_1.row == island1_2.row)
    bridge2_horizontal = (island2_1.row == island2_2.row)

    if bridge1_horizontal == bridge2_horizontal:
        return False  # Beide horizontal oder beide vertikal - keine Kreuzung

    # Je nachdem ob die erste Brücke horizontal oder vertikal ist, die Koordinaten setzen
    # und die Grenzen der anderen Brücke bestimmen
    if bridge1_horizontal:
        h_row = island1_1.row
        h_col_min, h_col_max = min(island1_1.col, island1_2.col), max(island1_1.col, island1_2.col)
        v_col = island2_1.col
        v_row_min, v_row_max = min(island2_1.row, island2_2.row), max(island2_1.row, island2_2.row)
    else:
        h_row = island2_1.row
        h_col_min, h_col_max = min(island2_1.col, island2_2.col), max(island2_1.col, island2_2.col)
        v_col = island1_1.col
        v_row_min, v_row_max = min(island1_1.row, island1_2.row), max(island1_1.row, island1_2.row)

    # Prüfen ob sie sich tatsächlich kreuzen
    return (h_col_min < v_col < h_col_max) and (v_row_min < h_row < v_row_max)

def no_bridge_crossing(assignment: Assignment, var: Var_PossibleBridge) -> bool:

    active_bridges = [v for v, val in assignment.items()
                     if val is not None and val > 0 and v != var]

    return all(not do_bridges_cross(var, other_var) for other_var in active_bridges)

### 3. Sicherheitsprüfung für ein Assignment

Überprüft ein Assignment und stellt sicher, dass die verbundenen Inseln noch erfüllbar bleiben und das Assignment keine sich kreuzenden Brücken erstellt.


In [41]:
def is_assignment_consistent(assignment: Assignment, var: Var_PossibleBridge, value: int) -> bool:

    # Temporäre Zuweisung erstellen
    temp_assignment = assignment.copy()
    temp_assignment[var] = value

    # Beide durch diese Variable verbundenen Inseln prüfen
    return (is_island_satisfiable(var[0], temp_assignment) and
            is_island_satisfiable(var[1], temp_assignment) and
            # Wenn mindestens eine Brücke gesetzt wird, verifizieren dass Brücke keine Kreuzung erstellt
            (value == 0 or no_bridge_crossing(temp_assignment, var)))


### 4. Gültige Domänenwerte ermitteln

Testet alle möglichen Werte (0,1,2) für eine Brücke und filtert ungültige Optionen heraus. Gibt möglicherweise eine leere Liste zurück, wenn keine gültigen Werte gefunden wurden. Die Rückgabe ist von groß nach klein sortiert.


In [42]:
def get_valid_domain_values(assignment: Assignment, var: Var_PossibleBridge) -> Iterable[int]:

    # Werte in Reihenfolge der Präferenz testen (höhere Werte zuerst)
    return filter(lambda v: is_assignment_consistent(assignment, var, v), reversed(domain))

### 5. Insel-Konnektivitätsprüfung

Verwendet BFS um zu prüfen ob alle Inseln über Brücken erreichbar sind. Wenn `islands=Nonde` (default), werden nur Inseln geprüft, die bereits Brücken haben.


In [43]:
def check_island_connectivity(assignment: Assignment, islands: Optional[Iterable[Island]] = None) -> bool:
    # Adjazenzgraph erstellen
    graph = defaultdict(list)
    if islands:
        relevant_islands = set(islands)
    else:
        relevant_islands = set()
    
    for (id1, id2), bridges in assignment.items():
        if bridges is not None and bridges > 0:
            if islands is None:
                relevant_islands.add(id1)
                relevant_islands.add(id2)
            graph[id1].append(id2)
            graph[id2].append(id1)

    # BFS zur Konnektivitätsprüfung
    start_island = next(iter(relevant_islands), None)
    visited = set([start_island])
    frontier = deque([start_island])

    while frontier and len(visited) != len(relevant_islands):
        current = frontier.popleft()
        for neighbor in graph[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                frontier.append(neighbor)

    return len(visited) == len(relevant_islands)


### 6. Lösungsvalidierung

Überprüft ob alle Insel-Constraints erfüllt sind und alle Inseln verbunden sind

In [44]:
def validate_complete_solution(board: HashiwokakeroBoard, assignment: Assignment) -> bool:

    # Prüfen dass alle Inseln exakte Brückenanzahl haben
    for island in board.islands:
        current_bridges = sum(val for var, val in assignment.items() if island in var and val is not None)
        if current_bridges != island.value:
            return False

    # Konnektivität für alle Inseln prüfen
    return check_island_connectivity(assignment, board.islands)


### 7. Lösung auf Spiel anwenden

Setzt das Spiel zurück und erstellt Brücken basierend auf der gegebenen Zuweisung


In [45]:
def apply_assignment_to_board(board: HashiwokakeroBoard, assignment: Assignment) -> None:

    board.bridges = []
    
    for (island1, island2), bridges in assignment.items():
        if bridges is not None and bridges > 0:
            board.add_bridge(island1, island2, bridges)


## Implementierung


### **(02.2.1)**: Variablen-Sortierung

*Für <mark>3</mark> Punkte*

Implementiert die Funktion `order_variables_by_constraint`, die die möglichen Variablen sortiert. Verwendet hierfür eine Kombination aus den folgenden Heuristiken:
- **MRV (Minimum Remaining Values) Heuristik**: Variablen mit **weniger gültigen Werten zuerst** bearbeitet werden sollten. 
- **Degree Heuristik**: am meisten eingeschränkte Inseln sollten zuerst betrachtet werden.

Ihr könnt natürlich auch weitere Hilfsfunktionen verwenden, um die Sortierung zu implementieren.

In [46]:
# Implementation Aufgabe 02.2.1

def order_variables_by_constraint(assignment: Assignment, variables: List[Var_PossibleBridge]) -> List[Var_PossibleBridge]:
    """_summary_

    Args:
        assignment (Assignment): current assignment of the variables
        variables (List[Var_PossibleBridge]): Unassigned variables to order	

    Returns:
        List[Var_PossibleBridge]: A list of variables ordered by the MRV and degree heuristic.
    """
    # TODO: Implement
    # Sortiere die Variablen nach der Anzahl der unzugewiesenen Brücken
    variables.sort(key=lambda var: (len(get_unassigned_vars(assignment, var[0])), 
                                     len(get_unassigned_vars(assignment, var[1]))), reverse=True)
    
    
    
    return variables 



#### Test für Aufgabe 02.2.1

In [47]:
def test_order_variables_by_constraint():

    island1 = Island(0, 0, 1)  # Wenig eingeschränkt
    island2 = Island(0, 1, 4)  # Sehr eingeschränkt
    island3 = Island(0, 2, 1)  # Wenig eingeschränkt
    island4 = Island(1, 1, 2)  # Mittel eingeschränkt
    assignment: Assignment = {
        (island1, island2): None,
        (island2, island3): None,
        (island2, island4): None
    }
    variables = list(assignment.keys())


    # Test ausführen
    ordered = order_variables_by_constraint(assignment, variables)

    print(f"Original variables:")
    pprint(variables)
    print(f"Ordered variables:")
    pprint(ordered)
    print("✓ Test completed - check if most constrained variables are first")
    print()

test_order_variables_by_constraint()

Original variables:
[(Island(row=0, col=1, value=4), Island(row=0, col=2, value=1)),
 (Island(row=0, col=1, value=4), Island(row=1, col=1, value=2)),
 (Island(row=0, col=0, value=1), Island(row=0, col=1, value=4))]
Ordered variables:
[(Island(row=0, col=1, value=4), Island(row=0, col=2, value=1)),
 (Island(row=0, col=1, value=4), Island(row=1, col=1, value=2)),
 (Island(row=0, col=0, value=1), Island(row=0, col=1, value=4))]
✓ Test completed - check if most constrained variables are first



### **(02.2.2)** Backtracking-Suche

*Für <mark>5</mark> Punkte*

Implementieren Sie die Funktion `backtrack_search`, die den Backtracking-Algorithmus für das CSP Problem umsetzt.

<details>
<summary>Tipps</summary>

- **Sortiert die unzugewiesenen Variablen** und wählt die erste Variable aus
- Probiert **jeden gültigen Wert** in Reihenfolge und startet eine rekursive Suche
- **Kopiert das `assignment`** vor jeder Veränderung

</details>

In [ ]:
def backtrack_search(board: HashiwokakeroBoard, assignment: Assignment,
                    timeout_time: float) -> Tuple[bool, Assignment]:

    if time.time() > timeout_time:
        return False, assignment

    # 1. Prüfen ob Zuweisung vollständig ist
    unassigned_vars = get_unassigned_vars(assignment)
    if not unassigned_vars:
        # Prüfe ob Lösung gültig ist
        if validate_complete_solution(board, assignment):
            return True, assignment
        else:
            return False, assignment

    # 2. Variablen sortieren (MRV + Degree)
    ordered_vars = order_variables_by_constraint(assignment, unassigned_vars)
    var = ordered_vars[0]

    # 3. Für jeden gültigen Wert rekursiv versuchen
    for value in get_valid_domain_values(assignment, var):
        new_assignment = assignment.copy()
        new_assignment[var] = value

        success, result_assignment = backtrack_search(board, new_assignment, timeout_time)
        if success:
            return True, result_assignment

    return False, assignment  


#### Test für 02.2.2

In [49]:
def test_backtrack_search():
        
    island1 = Island(0, 0, 2)
    island2 = Island(0, 1, 1) 
    island3 = Island(1, 0, 1)
    board = HashiwokakeroBoard(islands=[island1, island2, island3])
    
    assignment: Assignment = {
        (island1, island2): None,
        (island1, island3): None
    }
    timeout_time = time.time() + 10

    success, final_assignment = backtrack_search(board, assignment, timeout_time)
    print(f"Search successful: {success}")
    print(f"Final assignment:")
    pprint(final_assignment)
    print("✓ Test completed - check if backtracking logic works")

test_backtrack_search()

RecursionError: maximum recursion depth exceeded

### **(02.2.3)** Haupt-Lösungsfunktion
*Für <mark>3</mark> Punkte*

Implementieren Sie die Funktion `solve_hashiwokakero`, die den gesamten CSP-Solver orchestriert. Diese Funktion sollte alle vorher implementierten Funktionen aufrufen und die Lösung auf das Spielfeld anwenden, falls eine gefunden wurde.

<details>
<summary>Tipps</summary>

- **Erstellt ein initiales Assignment wo alle Variablen noch nicht zugewiesen sind** und nutzt dafür die Hilfsfunktion `board.find_potential_connections()`
- **Erstellt ein timeout** wann der solver abbrechen soll, um eine endlose Suche zu vermeiden (z.B. 60 Sekunden)
- **Ruft die Funktion `backtrack_search`** auf und übergibt das initiale Assignment sowie die maximale Laufzeit
- **Wenn eine Lösung gefunden wurde, wendet sie auf das Spielbrett an** mit der Funktion `apply_assignment_to_board`

</details>

In [ ]:
def solve_hashiwokakero(board: HashiwokakeroBoard) -> bool:
    print("Starting CSP solver...")

    # Initiales Assignment: alle möglichen Brücken auf None setzen
    variables = board.find_potential_connections()
    assignment: Assignment = {var: None for var in variables}

    # Timeout in 60 Sekunden
    timeout_time = time.time() + 60

    # Backtracking-Suche starten
    success, final_assignment = backtrack_search(board, assignment, timeout_time)

    if success:
        apply_assignment_to_board(board, final_assignment)
        return True

    return False


#### Test für Aufgabe 02.2.3

Folgendes Spielbrett wird für den Testaufruf verwendet:

![](https://upload.wikimedia.org/wikipedia/commons/d/d9/Ejemplo_hashiwokakero_5x5.PNG)

In [50]:
def test_solve_hashiwokakero(verbose=True):
    """Test der Hauptlösungsfunktion"""
    if verbose:
        print("=== Test: Main Solver Function ===")

    # Einfaches Test-Setup ohne pygame

    board = HashiwokakeroBoard.from_tuple_definition(
        [
            (0, 0, 1),
            (0, 1, 2),
            (0, 2, 1),
            (1, 0, 3),
            (1, 1, 8),
            (1, 2, 5),
            (2, 0, 2),
            (2, 1, 6),
            (2, 2, 4)
        ],
        grid_size=3
    )

    success = solve_hashiwokakero(board)

    print(f"Solver completed: {success}")
    print(f"Final bridges:")
    pprint(board.bridges)
    print("✓ Test completed - check if main solver coordinates all functions")
    print()


test_solve_hashiwokakero()

=== Test: Main Solver Function ===
Starting CSP solver...


RecursionError: maximum recursion depth exceeded

### Spielaufruf

Nun da ihr die Implementierung des CSP-Solvers abgeschlossen haben, könnt Ihr das Spiel aufrufen und die Puzzles automatisch lösen!

#### Controls:
- Drücke **'S'** um das aktuelle Puzzle zu lösen
- Drücke **'R'** um das Puzzle zurückzusetzen
- Drücke **'1'**, **'2'**, **'3'** um verschiedene Puzzles zu laden

In [ ]:
def main():
    """
    Vereinfachte Hauptfunktion für Spielausführung
    """
    # Spiel erstellen
    game = HashiwokakeroGameEngine(solve=solve_hashiwokakero)

    print("=== Hashiwokakero CSP Solver ===")
    print("Controls:")
    print("- Press 'S' to solve current puzzle")
    print("- Press 'R' to reset puzzle")
    print("- Press '1' to load puzzle 1")
    print("- Press '2' to load puzzle 2")
    print("- Press '3' to load puzzle 3")
    print("- Press ESC or close window to quit")
    print()
    
    game.run()


main()

=== Hashiwokakero CSP Solver ===
Controls:
- Press 'S' to solve current puzzle
- Press 'R' to reset puzzle
- Press '1' to load puzzle 1
- Press '2' to load puzzle 2
- Press '3' to load puzzle 3
- Press ESC or close window to quit

Starting CSP solver...
No solution found.


### **(02.2.4)** Schwachstellen der Implementierung
*Für <mark>4</mark> Punkte*

In dieser gesamten Aufgaben haben wir einen einfachen CSP-Solver für das Hashiwokakero-Puzzle implementiert. Die Umsetzung ist aber nicht unbedingt optimal und kann in einigen Fällen ineffizient sein. Das merkt ihr vermutlich auch, wenn ihr das `puzzle3` lösen lasst.

Nennt **zwei** mögliche Schwachstellen der Implementierung und Lösungsansätze um diese zu beheben.

> 1. Bei sehr großen Boards könnte es zu Problemen kommen, da das Programm in der rekursiven backtracking Suche die maximale Rekursionstiefe von Python übersteigt. Lösen könnte man das mit dem Ansatz, indem man das Puzzle in mehrere kleinere Teilpuzzles aufteilt und diese dann einzeln löst. 
> Hierbei muss dann aber berücksichtigt werden, dass das Puzzle durch manche sehr lange (mögliche) Brücken sehr kompliziert werden kann und ein Algorithmus zur Wahl von unabhängigen Teilpuzzles entsprechend kompliziert ist.
> 2. 

> 